In [1]:
# Clean up old CSV files and re-run
!rm actions_*.csv outcomes_*.csv;

rm: cannot remove 'actions_*.csv': No such file or directory
rm: cannot remove 'outcomes_*.csv': No such file or directory


In [ ]:
from battle import main
main(5)

  1%|          | 1/100 [01:43<2:50:51, 103.55s/it]

Epoch 0000:  0 ± 0 wins in 17.4 + 86.1 sec


  2%|▏         | 2/100 [02:16<1:41:14, 61.98s/it] 

Epoch 0001:  0 ± 0 wins in 17.5 + 15.4 sec


  3%|▎         | 3/100 [02:49<1:18:31, 48.58s/it]

...

Epoch 0026:  0 ± 0 wins in 294.1 + 70.0 sec


 28%|██▊       | 28/100 [46:27<6:15:51, 313.21s/it]

Epoch 0027:  0 ± 0 wins in 393.5 + 88.3 sec


 29%|██▉       | 29/100 [54:27<7:09:48, 363.22s/it]

Epoch 0028:  0 ± 0 wins in 380.7 + 99.1 sec


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotnine as pn
rnd = np.random.default_rng()

In [ ]:
def load(p):
    df = pd.read_csv(p)
    df['csv'] = str(p)
    return df

df = pd.concat([load(x) for x in Path().glob("outcomes_*.csv") if x.stat().st_size > 1000], ignore_index=True)
df.team = np.array(['PCs', 'monsters'])[df.team]
df.head()

How injured does each actor end up?

In [ ]:
x = df[df.epoch > df.epoch.max()-10].copy() # last 10 epochs
#x = x.groupby(['epoch', 'encounter', 'csv', 'team']).sum()
#x['frac_hp'] = x.final_hp / x.max_hp
x = x.reset_index()
#pn.qplot('max_hp', data=x, geom='histogram', fill='actor', binwidth=1) + pn.facet_wrap('actor')
pn.qplot('final_hp', data=x, geom='histogram', fill='actor', binwidth=1) + pn.facet_wrap('actor')

In [ ]:
def load(p):
    df = pd.read_csv(p)
    df['csv'] = str(p)
    return df

df = pd.concat([load(x) for x in Path().glob("actions_*.csv") if x.stat().st_size > 1000], ignore_index=True)
df.head(20)

How long do battles typically last, as a function of epoch?

In [ ]:
x = df.groupby(['epoch', 'encounter', 'csv'])['round'].max().groupby('epoch')
y = x.quantile(0.1)+1; plt.plot(y.index, y)
y = x.quantile(0.5)+1; plt.plot(y.index, y)
y = x.quantile(0.9)+1; plt.plot(y.index, y)

How often do we choose each possible action, as a function of time?

In [ ]:
pn.qplot('epoch', data=df, fill='action', geom='histogram', binwidth=1) + pn.facet_wrap('actor')

Close-up from final epoch:

In [ ]:
x = df[df.epoch == df.epoch.max()]
pn.qplot('action', data=x, fill='action', geom='histogram', binwidth=1) + pn.facet_wrap('actor') + pn.theme(axis_text_x=pn.element_text(rotation=90))

Do we use our healing potions more effectively as time goes on?

In [ ]:
# Mean effect?
x = df.query('raw_hp > 0').groupby('epoch')['obs_hp'].mean()
plt.plot(x.index, x)
# How often do we use it when we're uninjured?  Not often, it's forbidden.
# x = df.query('raw_hp > 0 and obs_hp == 0').groupby('epoch').count()
# plt.plot(x.index, x.obs_hp)

Which enemy does each character attack?

In [ ]:
x = df.query('raw_hp < 0')
pn.qplot('epoch', data=x, fill='target', geom='histogram', binwidth=1) + pn.facet_wrap('actor')

How often do we target the weakest enemy?

In [ ]:
x = df.query('raw_hp < 0')
pn.qplot('epoch', data=x, fill='t_weakest', geom='histogram', binwidth=1) + pn.facet_wrap('actor')

How often do we switch targets within an encounter?

In [ ]:
x = df.query('actor == "Hero" and action == "long sword"').copy()
#x['round2'] = x.groupby(['epoch', 'encounter', 'csv'])['round'].rank().astype('int')
# Convert target to numerical values
x['tgt'] = pd.Categorical(x.target).codes
# Detect changes in target
x['change_tgt'] = (x.groupby(['epoch', 'encounter', 'csv'])['tgt'].diff() != 0)
x.head()
#dir(x.tgt.cat)

In [ ]:
# Minimum is 2 (for a win):  initial attack, and switch to second target
y = x.groupby('epoch')['change_tgt'].sum() / 1000
plt.plot(y.index, y)

Which enemy do we attack first?

In [ ]:
y = x.query('round == 0').groupby('epoch')['tgt'].mean()
plt.plot(y.index, y)

Look at a random encounter from the final epoch of training...

In [ ]:
x = df[(df.epoch == df.epoch.max()) & (df.csv == rnd.choice(df.csv))]
x = x[x.encounter == rnd.choice(x.encounter)]
x

In [ ]:
# Wins and losses at 2nd level:  2 fighters and 2 wizards vs. X goblins
pc_win_rate = {
    5: 990/1000, # 1:30
    6: 941/1000, # 1:45
    7: 798/1000, # 2:10
    8: 507/1000,
    9: 276/1000, # 3:20
    10: 144/1000, # 4 hours to train
}
#plt.plot(pc_win_rate.keys(), pc_win_rate.values())
pn.qplot(list(pc_win_rate.keys()), list(pc_win_rate.values()), geom='line', xlab='# of goblins', ylab="PCs' win rate")